In [ ]:
import geopandas as gpd
import numpy as np

### Ellipse

In [ ]:
def make_ellipse(xy, w: float, h: float, *, angle: float = 0, half: bool = False):
    """
    Parameters
    ----------
    xy
        Center point.
    w, h
        Ellipse width and height.
        (Half-width and half-height, the semi-diameters, if `half` is True.)
    angle
        Rotation (counter-clockwise; applied after creation; **degrees**).
        For example ``angle=90`` will cause width and height to be switched in the result.
    half
        Whether `w` and `h` are expressed as 
        full diameters (``False``, default)
        or semi/half diameters (``True``).
    """
    # Based on https://gis.stackexchange.com/a/243462
    import shapely.affinity
    from shapely.geometry import Point

    if half:
        hw, hh = w, h
    else:
        hw, hh = w / 2, h / 2
    
    return shapely.affinity.rotate(shapely.affinity.scale(Point(xy).buffer(1), hw, hh), angle)

ell = make_ellipse((0, 0), 1, 1, angle=0, half=True)
print(ell.bounds)

ell = make_ellipse((-10, 5), 1.2, 0.8, angle=15)

display(ell)

gpd.GeoSeries(ell, crs="EPSG:4326").plot(fc="none")

## "Cone"

In [ ]:
def make_cone(x_range, y_range, r_range):
    """
    Parameters
    ----------
    x_range, y_range, r_range : tuple[float, float]
        Center and radius of initial and final circles.

    Returns
    -------
    Polygon
    """
    from shapely.geometry import MultiPolygon, Polygon
    from shapely.ops import unary_union

    # Based on https://gis.stackexchange.com/a/326692
    n = np.ceil(max(np.ptp(x_range), np.ptp(y_range))).astype(int) * 2
    x = np.linspace(*x_range, n)
    y = np.linspace(*y_range, n)
    r = np.linspace(*r_range, n)

    # Coords
    theta = np.linspace(0, 2 * np.pi, 360)
    polygon_x = x[:, None] + r[:, None] * np.sin(theta)
    polygon_y = y[:, None] + r[:, None] * np.cos(theta)

    # Circles
    ps = [Polygon(i) for i in np.dstack((polygon_x, polygon_y))]

    # Convex hulls of subsequent circles
    n = range(len(ps)-1)
    convex_hulls = [MultiPolygon([ps[i], ps[i+1]]).convex_hull for i in n]

    # Final polygon
    cone = unary_union(convex_hulls)

    return cone


x_range = (0, -10)
y_range = (0, 0)
r_range = (0.2, 3)

cone = make_cone(x_range, y_range, r_range)

display(cone)

gpd.GeoSeries(cone, crs="EPSG:4326").plot(fc="none")

In [ ]:
# cones = cen.apply(lambda p: make_cone((p.x, p.x + dx), (p.y, p.y + dy), r_range))

In [ ]:
def make_line(p1, p2):
    """Return the line connecting the two points."""
    from shapely.geometry import LineString

    return LineString([p1, p2])

make_line((0, 0), (1, 2))

In [ ]:
def make_arc(xy, r: float, angle_range, *, n: int | None = None):
    """Make an arc line.

    Angles in `angle_range` are mapped to [0, 360) with ``%``,
    i.e. these inputs will have the same result:

    - (0, 180)
    - (0, -180)
    - (0, 540)

    Parameters
    ----------
    n
    """
    # Based on https://stackoverflow.com/a/30762727
    from shapely.geometry import Point, LineString

    p = Point(xy)

    a, b = angle_range
    a %= 360
    b %= 360
    
    if n is None:
        n = np.clip(abs(b - a), 10, 360)
    
    theta = np.deg2rad(np.linspace(a, b, n))
    x = p.x + r * np.cos(theta)
    y = p.y + r * np.sin(theta)
    
    return LineString(np.column_stack((x, y)))

make_arc((0, 0), 1, (150, 210))

In [ ]:
from shapely.geometry import MultiPolygon

def cut(poly, line):  # TODO: `split` better name?
    """Split polygon along line."""
    # Based on https://kuanbutts.com/2020/07/07/subdivide-polygon-with-linestring/
    from shapely.ops import polygonize
    
    to_cut = poly
    cutter = line
    
    # Union the exterior lines of the polygon with the dividing linestring
    unioned = to_cut.boundary.union(cutter)
    
    # Use polygonize geos operator and filter out poygons ouside of original input polygon
    keep_polys = [poly for poly in polygonize(unioned) if poly.representative_point().within(to_cut)]

    # Remaining polygons are the split polys of original shape
    return keep_polys

polys = cut(cone, make_arc((-3, 0), 4, (45, 360 - 45)))
print(len(polys))

display(MultiPolygon(polys))

gpd.GeoSeries(polys).plot(cmap="tab10")

### Split

## Circle

In [ ]:
def make_circle(xy, r):
    from shapely.geometry import Point

    return Point(xy).buffer(r/2)

cir = make_circle((0, 0), 1)

print(cir.bounds)

display(cir)

### Quadrants

In [ ]:
from itertools import chain

xy = (0, 0)

outer = make_circle(xy, 5)
inner = make_circle(xy, 2.5)

# y=x and y=-x
x = 10
line1 = make_line((-x, -x), (x, x))
line2 = make_line((-x, x), (x, -x))

cutters = [inner, line1, line2]

polys = [outer]
for cutter in cutters:
    polys = list(chain.from_iterable(cut(p, cutter) for p in polys))

print(len(polys))

s = gpd.GeoSeries(polys)
assert np.isclose(s.area.sum(), outer.area)
s.plot(cmap="tab10", lw=2)

In [ ]:
s.difference(make_ellipse((0, 0), 1.5, 0.8, angle=15)).plot(cmap="tab10")

In [ ]:
# TODO: rotation arg?
# TODO: elliptical version of this?
# TODO: split cone forward vs backward (with rotation) example?

## Rectangle

In [ ]:
def make_rectangle(ll, ur):
    """Return the rectangle defined by two points (lower-left and upper-right corner)."""
    from shapely.geometry import Point, Polygon
    
    ll, ur = Point(ll), Point(ur)
    if ll.x == ur.x or ll.y == ur.y:
        raise ValueError("not diagonal")

    return Polygon([(ll.x, ll.y), (ll.x, ur.y), (ur.x, ur.y), (ur.x, ll.y)])

rect = make_rectangle((0, 0), (4, 2))

display(rect)

gpd.GeoSeries(rect).plot()